# Web Scraping con Beautiful Soup.

* * * 

### Iconos utilizados en este cuaderno
🔔 **Pregunta**: Una pregunta rápida para ayudarte a entender lo que está pasando.<br>
🥊 **Desafío**: Ejercicio interactivo. ¡Trabajaremos en estos durante el taller!<br>
⚠️ **Advertencia**: Aviso sobre cosas complicadas o errores comunes.<br>
💡 **Consejo**: Cómo hacer algo de manera más eficiente o efectiva.<br>
🎬 **Demostración**: Mostrar algo más avanzado, para que sepas para qué se puede usar Python.<br>

### Objetivos de aprendizaje
1. [Reflexión: ¿Raspado o no raspado?](#when)
2. [Extracción y análisis de HTML](#extract)
3. [Raspado de la Asamblea General de Illinois](#scrape)


<a id='when'></a>

# Scrape o no Scrape

Cuando queremos acceder a datos de la web, primero debemos asegurarnos de si el sitio web que nos interesa ofrece una API web. Plataformas como Twitter, Reddit y el New York Times ofrecen APIs. **Consulta el taller de APIs web de [Python de D-Labs](https://github.com/dlab-berkeley/Python-Web-APIs) si quieres aprender a usar APIs.**

Sin embargo, a menudo hay casos en los que no existe una API web. En estos casos, podemos recurrir al web scraping, donde extraemos el HTML subyacente de una página web y obtenemos directamente la información que queremos. Hay varios paquetes en Python que podemos usar para realizar estas tareas. Nos centraremos en dos paquetes: Requests y Beautiful Soup.

Nuestro estudio de caso será raspar información sobre los [senadores estatales de Illinois](http://www.ilga.gov/senate), así como la [lista de proyectos de ley](http://www.ilga.gov/senate/SenatorBills.asp?MemberID=1911&GA=98&Primary=True) que cada senador ha patrocinado. Antes de comenzar, revisa estos sitios web para ver su estructura.




## Instalación

Usaremos dos paquetes principales: [Requests](http://docs.python-requests.org/en/latest/user/quickstart/) y [Beautiful Soup](http://www.crummy.com/software/BeautifulSoup/bs4/doc/). Adelante, instala estos paquetes si aún no lo has hecho:


In [1]:
%pip install requests

Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install beautifulsoup4

Note: you may need to restart the kernel to use updated packages.


También instalaremos el paquete `lxml` que ayuda a soportar parte del análisis que realiza Beautiful Soup:

In [3]:
%pip install lxml

Note: you may need to restart the kernel to use updated packages.


In [4]:
# Import required libraries
from bs4 import BeautifulSoup
from datetime import datetime
import requests
import time

<a id='extract'></a>

# Extracción y Análisis de HTML 

Para realizar correctamente la extracción y el análisis de HTML, seguiremos los siguientes 4 pasos:
1. Realizar una solicitud GET
2. Analizar la página con Beautiful Soup
3. Buscar elementos HTML
4. Obtener atributos y texto de estos elementos

## Paso 1: Realizar una Solicitud GET para Obtener el HTML de una Página

Podemos usar la biblioteca Requests para:

1. Hacer una solicitud GET a la página, y  
2. Leer el código HTML de la página web.

El proceso de realizar una solicitud y obtener un resultado se asemeja al flujo de trabajo de una API web. Sin embargo, en este caso, estamos haciendo una solicitud directamente al sitio web, y tendremos que analizar el HTML por nuestra cuenta. Esto contrasta con recibir datos organizados en un formato más sencillo como `JSON` o `XML`.


In [109]:
# Make a GET request
req = requests.get('https://www.ilga.gov/house/default.asp')
# Read the content of the server’s response
src = req.text
# Parse the response into an HTML tree
soup = BeautifulSoup(src, 'lxml')

## Paso 2: Analizar la Página con Beautiful Soup

Ahora utilizamos la función `BeautifulSoup` para analizar la respuesta y convertirla en un árbol HTML. Esto devuelve un objeto (llamado **soup object**) que contiene todo el HTML del documento original.

Si encuentras un error relacionado con una biblioteca de análisis, asegúrate de haber instalado el paquete `lxml` para proporcionar a Beautiful Soup las herramientas necesarias para el análisis.

In [110]:
# Parse the response into an HTML tree
soup = BeautifulSoup(src, 'lxml')
# Take a look
print(soup.prettify()[:1000])

<html lang="en">
 <!-- Trigger/Open The Modal -->
 <body>
  <div style="position: fixed; z-index: 999; top: 5; left: 600; background-color: navy; display: block">
   <button id="myBtn" style="color: white; background-color: navy; display: block">
    Translate Website
   </button>
  </div>
  <!-- The Modal -->
  <div class="modal" id="myModal" style="display: none">
   <!-- Modal content -->
   <div class="modal-content">
    <div class="modal-header">
     <h3>
      <span class="close">
       ×
      </span>
     </h3>
    </div>
    <p>
     The Illinois General Assembly offers the Google Translate service for visitor convenience. In no way should it be considered accurate as to the translation of any content herein.
    </p>
    <p>
     Visitors of the Illinois General Assembly website are encouraged to use other translation services available on the internet.
    </p>
    <p>
     The English language version is always the official and authoritative version of this website.
   

La salida se ve bastante similar a la anterior, pero ahora está organizada en un objeto `soup`, lo que nos permite recorrer la página con mayor facilidad.


## Paso 3: Buscar Elementos HTML

Beautiful Soup ofrece varias funciones para encontrar componentes útiles en una página. Permite buscar elementos utilizando:

1. Etiquetas HTML  
2. Atributos HTML  
3. Selectores CSS  

Primero, busquemos **etiquetas HTML**.  

La función `find_all` busca en el árbol de `soup` todos los elementos con una etiqueta HTML específica y devuelve todos esos elementos.

**¿Qué hace el siguiente ejemplo?**



In [111]:
# Find all elements with a certain tag
a_tags = soup.find_all("a")
print(a_tags[:10])

[<a class="goog-logo-link" href="https://translate.google.com" target="_blank"><img alt="Google Translate" height="14" src="https://www.gstatic.com/images/branding/googlelogo/1x/googlelogo_color_42x16dp.png" style="padding-right: 3px;" width="37"/>Translate</a>, <a href="/default.asp"><img alt="Illinois General Assembly" border="0" height="49" src="/images/logo_sm.gif" width="462"/></a>, <a class="mainmenu" href="/">Home</a>, <a class="mainmenu" href="/legislation/" onblur="HM_f_PopDown('elMenu1')" onfocus="HM_f_PopUp('elMenu1',event)" onmouseout="HM_f_PopDown('elMenu1')" onmouseover="HM_f_PopUp('elMenu1',event)">Legislation &amp; Laws</a>, <a class="mainmenu" href="/senate/" onblur="HM_f_PopDown('elMenu3')" onfocus="HM_f_PopUp('elMenu3',event)" onmouseout="HM_f_PopDown('elMenu3')" onmouseover="HM_f_PopUp('elMenu3',event)">Senate</a>, <a class="mainmenu" href="/house/" onblur="HM_f_PopDown('elMenu2')" onfocus="HM_f_PopUp('elMenu2',event)" onmouseout="HM_f_PopDown('elMenu2')" onmouseove

Dado que `find_all()` es el método más popular en la API de búsqueda de Beautiful Soup, existe un atajo para usarlo. Si tratas al objeto `BeautifulSoup` como si fuera una función, es equivalente a llamar a `find_all()` en ese objeto.

Estas dos líneas de código son equivalentes:

In [112]:
a_tags = soup.find_all("a")
a_tags_alt = soup("a")
print(a_tags[0])
print(a_tags_alt[0])

<a class="goog-logo-link" href="https://translate.google.com" target="_blank"><img alt="Google Translate" height="14" src="https://www.gstatic.com/images/branding/googlelogo/1x/googlelogo_color_42x16dp.png" style="padding-right: 3px;" width="37"/>Translate</a>
<a class="goog-logo-link" href="https://translate.google.com" target="_blank"><img alt="Google Translate" height="14" src="https://www.gstatic.com/images/branding/googlelogo/1x/googlelogo_color_42x16dp.png" style="padding-right: 3px;" width="37"/>Translate</a>


¿Cuántos enlaces obtuvimos?

In [113]:
print(len(a_tags))

382


¡Eso es mucho! Muchos elementos en una página tendrán la misma etiqueta HTML. Por ejemplo, si buscas todo lo que tenga la etiqueta `a`, probablemente obtendrás muchos resultados, muchos de los cuales quizás no quieras. Recuerda que la etiqueta `a` define un hipervínculo, por lo que generalmente encontrarás muchos en cualquier página.

¿Qué pasa si queremos buscar etiquetas HTML con ciertos atributos, como clases CSS específicas?

Podemos hacerlo agregando un argumento adicional a la función `find_all`. En el ejemplo siguiente, estamos encontrando todas las etiquetas `a` y luego filtrando aquellas con `class_="sidemenu"`:


In [114]:
# Get only the 'a' tags in 'sidemenu' class
side_menus = soup("a", class_="sidemenu")
side_menus[:5]

[<a class="sidemenu" href="/house/default.asp">  Members  </a>,
 <a class="sidemenu" href="/house/committees/default.asp">  Committees  </a>,
 <a class="sidemenu" href="/house/schedules/default.asp">  Schedules  </a>,
 <a class="sidemenu" href="/house/journals/default.asp">  Journals  </a>,
 <a class="sidemenu" href="/house/transcripts/default.asp">  Transcripts  </a>]

Una forma más eficiente de buscar elementos en un sitio web es a través de un **selector CSS**. Para esto, debemos usar un método diferente llamado `select()`. Solo tienes que pasar una cadena al método `.select()` para obtener todos los elementos que coincidan con ese selector CSS válido.

En el ejemplo anterior, podemos usar `"a.sidemenu"` como selector CSS, lo que devuelve todas las etiquetas `a` con la clase `sidemenu`.



In [115]:
# Get elements with "a.sidemenu" CSS Selector.
selected = soup.select("a.sidemenu")
selected[:5]

[<a class="sidemenu" href="/house/default.asp">  Members  </a>,
 <a class="sidemenu" href="/house/committees/default.asp">  Committees  </a>,
 <a class="sidemenu" href="/house/schedules/default.asp">  Schedules  </a>,
 <a class="sidemenu" href="/house/journals/default.asp">  Journals  </a>,
 <a class="sidemenu" href="/house/transcripts/default.asp">  Transcripts  </a>]

## 🥊 Desafío: Encontrar Todos

Usa BeautifulSoup para encontrar todos los elementos `a` con la clase `mainmenu`.

In [116]:
# YOUR CODE HERE
soup.select("a.mainmenu")

[<a class="mainmenu" href="/">Home</a>,
 <a class="mainmenu" href="/legislation/" onblur="HM_f_PopDown('elMenu1')" onfocus="HM_f_PopUp('elMenu1',event)" onmouseout="HM_f_PopDown('elMenu1')" onmouseover="HM_f_PopUp('elMenu1',event)">Legislation &amp; Laws</a>,
 <a class="mainmenu" href="/senate/" onblur="HM_f_PopDown('elMenu3')" onfocus="HM_f_PopUp('elMenu3',event)" onmouseout="HM_f_PopDown('elMenu3')" onmouseover="HM_f_PopUp('elMenu3',event)">Senate</a>,
 <a class="mainmenu" href="/house/" onblur="HM_f_PopDown('elMenu2')" onfocus="HM_f_PopUp('elMenu2',event)" onmouseout="HM_f_PopDown('elMenu2')" onmouseover="HM_f_PopUp('elMenu2',event)">House</a>,
 <a class="mainmenu" href="/mylegislation/" onblur="HM_f_PopDown('elMenu4')" onfocus="HM_f_PopUp('elMenu4',event)" onmouseout="HM_f_PopDown('elMenu4')" onmouseover="HM_f_PopUp('elMenu4',event)">My Legislation</a>,
 <a class="mainmenu" href="/sitemap.asp">Site Map</a>]

In [117]:
[link['href'] for link in soup.select("a.mainmenu")]

['/',
 '/legislation/',
 '/senate/',
 '/house/',
 '/mylegislation/',
 '/sitemap.asp']

## Paso 4: Obtener Atributos y Texto de los Elementos

Una vez que identificamos los elementos, queremos acceder a la información dentro de ellos. Generalmente, esto significa dos cosas:

1. Texto  
2. Atributos  

Obtener el texto dentro de un elemento es fácil. Todo lo que tenemos que hacer es usar el miembro `text` de un objeto `tag`:

In [118]:
# Get all sidemenu links as a list
side_menu_links = soup.select("a.sidemenu")

# Examine the first link
first_link = side_menu_links[0]
print(first_link)

# What class is this variable?
print('Class: ', type(first_link))

<a class="sidemenu" href="/house/default.asp">  Members  </a>
Class:  <class 'bs4.element.Tag'>


¡Es una etiqueta de Beautiful Soup! Esto significa que tiene un miembro `text`:

In [18]:
print(first_link.text)

  Members  


A veces queremos obtener el valor de ciertos atributos. Esto es particularmente relevante para las etiquetas `a`, o enlaces, donde el atributo `href` nos indica a dónde lleva el enlace.

💡 **Consejo**: Puedes acceder a los atributos de una etiqueta tratándola como un diccionario:

In [19]:
print(first_link['href'])

/senate/default.asp


## 🥊 Desafío: Extraer atributos específicos

Extrae todos los atributos `href` de cada URL con la clase `mainmenu`.

In [20]:
# YOUR CODE HERE
[link['href'] for link in soup.select("a.mainmenu")]

['/',
 '/legislation/',
 '/senate/',
 '/house/',
 '/mylegislation/',
 '/sitemap.asp']

<a id='scrape'></a>

# Arrastrando la Asamblea General de Illinois
Aunque parezca increíble, estas son las herramientas fundamentales para arrastrar un sitio web. Una vez que dediques más tiempo a familiarizarte con HTML y CSS, simplemente será cuestión de comprender la estructura de un sitio web en particular y aplicar inteligentemente las herramientas de Beautiful Soup y Python.

Apliquemos estas habilidades para arrastrar la [98.ª Asamblea General de Illinois](http://www.ilga.gov/senate/default.asp?GA=98)..

En concreto, nuestro objetivo es arrastrar información sobre cada senador, incluyendo su nombre, distrito y partido.

## Arrastrar y Disolver la página web

Arrastrar y Analizar la página web con las herramientas que aprendimos en la sección anterior.

In [119]:
# Make a GET request
req = requests.get('https://www.ilga.gov/house/default.asp?GA=104')
# Read the content of the server’s response
src = req.text
# Soup it
soup = BeautifulSoup(src, "lxml")

## Buscar los elementos de la tabla

Nuestro objetivo es obtener los elementos de la tabla en la página web. Recuerda: las filas se identifican con la etiqueta `tr`. Usaremos `find_all` para obtener estos elementos.

In [120]:
# Get all table row elements
rows = soup.find_all("tr")
len(rows)

136

⚠️ **Advertencia**: Recuerda: `find_all` obtiene todos los elementos con la etiqueta `tr`. Solo necesitamos algunos. Si usamos la función "Inspeccionar" en Google Chrome y observamos con atención, podemos usar selectores CSS para obtener solo las filas que nos interesan. En concreto, queremos las filas internas de la tabla:

In [129]:
# Returns every ‘tr tr tr’ css selector in the page
rows = soup.select('tr tr tr')

for row in rows[:5]:
    print(row, '\n')

<tr><td colspan="5">
<span class="heading">Current House Members</span>
<span class="italics">  104th General Assembly</span><br/>
<!--Removed link for Current only temporarily at rollover- please leave Previous check in so at next rollover it still shows for previous ga-->
<span class="content"> <a href="104th_House_Officers.pdf">Officers of the General Assembly</a></span><br/><span class="content"><b>Democrats:</b> 78   <b>Republicans:</b> 40</span><br/>
</td></tr> 

<tr>
<td class="header" width="45%"><a class="filetab" href="javascript:Sort('LastName','',104);" title="Sort by Representative">Representative</a></td>
<td align="center" class="header" width="15%">Bills</td>
<td align="center" class="header" width="10%">Committees</td>
<td align="center" class="header" width="15%"><a class="filetab" href="javascript:Sort('DistrictNumber','',104);" title="Sort by District">District</a></td>
<td align="center" class="header" width="15%"><a class="filetab" href="javascript:Sort('Party',''

Parece que queremos todo lo que queda después de las dos primeras filas. Empecemos con una sola fila y construyamos nuestro bucle a partir de ahí.

In [130]:
example_row = rows[2]
print(example_row.prettify())

<tr>
 <td bgcolor="white" class="detail" width="40%">
  <a class="notranslate" href="/house/Rep.asp?GA=104&amp;MemberID=3309">
   Carol Ammons
  </a>
 </td>
 <td align="center" bgcolor="white" class="detail" width="15%">
  <a href="RepBills.asp?MemberID=3309">
   Bills
  </a>
 </td>
 <td align="center" bgcolor="white" class="detail" width="15%">
  <a href="RepCommittees.asp?MemberID=3309">
   Committees
  </a>
 </td>
 <td align="center" bgcolor="white" class="notranslate detail" width="15%">
  103
 </td>
 <td align="center" bgcolor="white" class="notranslate detail" width="15%">
  D
 </td>
</tr>



Desglosemos esta fila en sus celdas/columnas mediante el método `select` con los métodos CSS. Si analizamos el HTML con atención, hay un par de maneras de hacerlo.

* Podríamos identificar las celdas por su etiqueta `td`.
* Podríamos usar el nombre de clase `.detail`.
* Podríamos combinar ambos y usar el selector `td.detail`.

In [131]:
for cell in example_row.select('td'):
    print(cell)
print()

for cell in example_row.select('.detail'):
    print(cell)
print()

for cell in example_row.select('td.detail'):
    print(cell)
print()

<td bgcolor="white" class="detail" width="40%"><a class="notranslate" href="/house/Rep.asp?GA=104&amp;MemberID=3309">Carol Ammons</a></td>
<td align="center" bgcolor="white" class="detail" width="15%"><a href="RepBills.asp?MemberID=3309">Bills</a></td>
<td align="center" bgcolor="white" class="detail" width="15%"><a href="RepCommittees.asp?MemberID=3309">Committees</a></td>
<td align="center" bgcolor="white" class="notranslate detail" width="15%">103</td>
<td align="center" bgcolor="white" class="notranslate detail" width="15%">D</td>

<td bgcolor="white" class="detail" width="40%"><a class="notranslate" href="/house/Rep.asp?GA=104&amp;MemberID=3309">Carol Ammons</a></td>
<td align="center" bgcolor="white" class="detail" width="15%"><a href="RepBills.asp?MemberID=3309">Bills</a></td>
<td align="center" bgcolor="white" class="detail" width="15%"><a href="RepCommittees.asp?MemberID=3309">Committees</a></td>
<td align="center" bgcolor="white" class="notranslate detail" width="15%">103</td

Podemos confirmar que todas son iguales.

In [132]:
assert example_row.select('td') == example_row.select('.detail') == example_row.select('td.detail')

Utilicemos el selector `td.detail` para ser lo más específicos posibles.

In [125]:
# Select only those 'td' tags with class 'detail' 
detail_cells = example_row.select('td.detail')
detail_cells

[<td bgcolor="white" class="detail" width="40%"><a class="notranslate" href="/house/Rep.asp?GA=104&amp;MemberID=3309">Carol Ammons</a></td>,
 <td align="center" bgcolor="white" class="detail" width="15%"><a href="RepBills.asp?MemberID=3309">Bills</a></td>,
 <td align="center" bgcolor="white" class="detail" width="15%"><a href="RepCommittees.asp?MemberID=3309">Committees</a></td>,
 <td align="center" bgcolor="white" class="notranslate detail" width="15%">103</td>,
 <td align="center" bgcolor="white" class="notranslate detail" width="15%">D</td>]

La mayoría de las veces, nos interesa el **texto** real de un sitio web, no sus etiquetas. Recordemos que para obtener el texto de un elemento HTML, usamos el `text` member:

In [133]:
# Keep only the text in each of those cells
row_data = [cell.text for cell in detail_cells]

print(row_data)

['Carol Ammons', 'Bills', 'Committees', '103', 'D']


¡Se ve bien! Ahora solo necesitamos usar nuestros conocimientos básicos de Python para obtener los elementos de esta lista que necesitamos. Recuerda: queremos el nombre del senador, su distrito y su partido.

In [134]:
print(row_data[0]) # Name
print(row_data[1]) # Bills
print(row_data[2]) # Committees
print(row_data[3]) # District
print(row_data[4]) # Party

Carol Ammons
Bills
Committees
103
D


## Eliminando las filas basura

Vimos al principio que no todas las filas que obtuvimos corresponden a un senador. Tendremos que hacer limpieza antes de continuar. Veamos algunos ejemplos:

In [135]:
print('Row 0:\n', rows[0], '\n')
print('Row 1:\n', rows[1], '\n')
print('Last Row:\n', rows[-1])

Row 0:
 <tr><td colspan="5">
<span class="heading">Current House Members</span>
<span class="italics">  104th General Assembly</span><br/>
<!--Removed link for Current only temporarily at rollover- please leave Previous check in so at next rollover it still shows for previous ga-->
<span class="content"> <a href="104th_House_Officers.pdf">Officers of the General Assembly</a></span><br/><span class="content"><b>Democrats:</b> 78   <b>Republicans:</b> 40</span><br/>
</td></tr> 

Row 1:
 <tr>
<td class="header" width="45%"><a class="filetab" href="javascript:Sort('LastName','',104);" title="Sort by Representative">Representative</a></td>
<td align="center" class="header" width="15%">Bills</td>
<td align="center" class="header" width="10%">Committees</td>
<td align="center" class="header" width="15%"><a class="filetab" href="javascript:Sort('DistrictNumber','',104);" title="Sort by District">District</a></td>
<td align="center" class="header" width="15%"><a class="filetab" href="javascript

Al escribir nuestro bucle for, queremos que solo se aplique a las filas relevantes. Por lo tanto, debemos filtrar las filas irrelevantes. Para ello, comparamos algunas de estas con las filas que necesitamos, observamos sus diferencias y luego formulamos esto en una condición.

Como puedes imaginar, hay muchas maneras de hacerlo, y dependerá del sitio web. Aquí te mostraremos algunas para que te hagas una idea de cómo hacerlo.

In [136]:
# Bad rows
print(len(rows[0]))
print(len(rows[1]))

# Good rows
print(len(rows[2]))
print(len(rows[3]))

1
11
5
5


Quizás las filas buenas tengan una longitud de 5. Comprobémoslo:

In [137]:
good_rows = [row for row in rows if len(row) == 5]

# Let's check some rows
print(good_rows[0], '\n')
print(good_rows[-2], '\n')
print(good_rows[-1])

<tr><td bgcolor="white" class="detail" width="40%"><a class="notranslate" href="/house/Rep.asp?GA=104&amp;MemberID=3309">Carol Ammons</a></td><td align="center" bgcolor="white" class="detail" width="15%"><a href="RepBills.asp?MemberID=3309">Bills</a></td><td align="center" bgcolor="white" class="detail" width="15%"><a href="RepCommittees.asp?MemberID=3309">Committees</a></td><td align="center" bgcolor="white" class="notranslate detail" width="15%">103</td><td align="center" bgcolor="white" class="notranslate detail" width="15%">D</td></tr> 

<tr><td bgcolor="EBEBEB" class="detail" width="40%"><a class="notranslate" href="/house/Rep.asp?GA=104&amp;MemberID=3387">Janet Yang Rohr</a></td><td align="center" bgcolor="EBEBEB" class="detail" width="15%"><a href="RepBills.asp?MemberID=3387">Bills</a></td><td align="center" bgcolor="EBEBEB" class="detail" width="15%"><a href="RepCommittees.asp?MemberID=3387">Committees</a></td><td align="center" bgcolor="EBEBEB" class="notranslate detail" width

Encontramos una fila de pie de página en nuestra lista que queremos evitar. Probemos algo diferente:

In [138]:
rows[2].select('td.detail') 

[<td bgcolor="white" class="detail" width="40%"><a class="notranslate" href="/house/Rep.asp?GA=104&amp;MemberID=3309">Carol Ammons</a></td>,
 <td align="center" bgcolor="white" class="detail" width="15%"><a href="RepBills.asp?MemberID=3309">Bills</a></td>,
 <td align="center" bgcolor="white" class="detail" width="15%"><a href="RepCommittees.asp?MemberID=3309">Committees</a></td>,
 <td align="center" bgcolor="white" class="notranslate detail" width="15%">103</td>,
 <td align="center" bgcolor="white" class="notranslate detail" width="15%">D</td>]

In [139]:
# Bad row
print(rows[-1].select('td.detail'), '\n')

# Good row
print(rows[5].select('td.detail'), '\n')

# How about this?
good_rows = [row for row in rows if row.select('td.detail')]

print("Checking rows...\n")
print(good_rows[0], '\n')
print(good_rows[-1])

[] 

[<td bgcolor="EBEBEB" class="detail" width="40%"><a class="notranslate" href="/house/Rep.asp?GA=104&amp;MemberID=3432">Harry Benton</a></td>, <td align="center" bgcolor="EBEBEB" class="detail" width="15%"><a href="RepBills.asp?MemberID=3432">Bills</a></td>, <td align="center" bgcolor="EBEBEB" class="detail" width="15%"><a href="RepCommittees.asp?MemberID=3432">Committees</a></td>, <td align="center" bgcolor="EBEBEB" class="notranslate detail" width="15%">97</td>, <td align="center" bgcolor="EBEBEB" class="notranslate detail" width="15%">D</td>] 

Checking rows...

<tr><td bgcolor="white" class="detail" width="40%"><a class="notranslate" href="/house/Rep.asp?GA=104&amp;MemberID=3309">Carol Ammons</a></td><td align="center" bgcolor="white" class="detail" width="15%"><a href="RepBills.asp?MemberID=3309">Bills</a></td><td align="center" bgcolor="white" class="detail" width="15%"><a href="RepCommittees.asp?MemberID=3309">Committees</a></td><td align="center" bgcolor="white" class="notr

¡Parece que encontramos algo que funcionó!

## Enlazar todo

Ahora que hemos visto cómo obtener los datos que queremos de una fila, así como filtrar las filas que no queremos, vamos a juntarlo todo en un bucle.

In [140]:
# Define storage list
members = []

# Get rid of junk rows
valid_rows = [row for row in rows if row.select('td.detail')]

# Loop through all rows
for row in valid_rows:
    # Select only those 'td' tags with class 'detail'
    detail_cells = row.select('td.detail')
    # Keep only the text in each of those cells
    row_data = [cell.text for cell in detail_cells]
    # Collect information
    name = row_data[0]
    committees = row_data[2]	
    district = int(row_data[3])
    party = row_data[4]
    # Store in a tuple
    senator = (name, committees, district, party)
    # Append to list
    members.append(senator)

In [143]:
print(members[:5])

[('Carol Ammons', 'Committees', 103, 'D'), ('Jaime M. Andrade, Jr.', 'Committees', 40, 'D'), ('Dagmara Avelar', 'Committees', 85, 'D'), ('Harry Benton', 'Committees', 97, 'D'), ('Diane Blair-Sherlock', 'Committees', 46, 'D')]


In [152]:
# Should be 61
total_mimbros = len(members)
print(total_mimbros)

118


## **CREAMOS LA BASE DE DATOS PARA INGRESAR LA DATA**

In [153]:
import psycopg2
import json
from datetime import datetime

# Paso 1: Crear la base de datos y la tabla desde Python
def create_database_and_table():
    try:
        # Conexión al servidor PostgreSQL (sin especificar una base de datos)
        conn = psycopg2.connect(
            user="postgres", # Usuario predeterminado
            password="postgres",
            host="localhost",
            port="5432"
        )
        conn.autocommit = True # Necesario para crear una base de datos
        cursor = conn.cursor()

        # Crear la base de datos "SENADORES_HOUSE_logs" si no existe
        cursor.execute("SELECT datname FROM pg_database WHERE datname='senadores_house_logs';")
        if not cursor.fetchone():
            cursor.execute("CREATE DATABASE senadores_house_logs;")
            print("Base de datos 'senadores_house_logs' creada correctamente.")
        else:
            print("La base de datos 'senadores_house_logs' ya existe.")

        # Cerrar la conexión inicial
        cursor.close()
        conn.close()

        # Conectar a la nueva base de datos
        conn = psycopg2.connect(
        database="senadores_house_logs",
        user="postgres",
        password="postgres",
        host="localhost",
        port="5432"
        )
        cursor = conn.cursor()

        cursor.execute("DROP TABLE IF EXISTS structured_logs;")

        # Crear la tabla "structured_logs" si no existe
        cursor.execute("""
        CREATE TABLE IF NOT EXISTS structured_logs (
        id SERIAL PRIMARY KEY,
        name VARCHAR(100),
        committees VARCHAR(250),
        distrit INTEGER,
        party TEXT
        );
        """)
        print("Tabla 'structured_logs' creada correctamente.")

        # Confirmar cambios y cerrar la conexión
        conn.commit()
        cursor.close()
        conn.close()

    except Exception as e:
        print(f"Error al crear la base de datos o la tabla: {e}")

# Paso 2: Insertar datos estructurados en la tabla
def insert_structured_logs():
    try:
        # Conectar a la base de datos
        conn = psycopg2.connect(
        database="senadores_house_logs",
        user="postgres",
        password="postgres",
        host="localhost",
        port="5432"
        )
        cursor = conn.cursor()        

        # Insertar registros en la tabla (máximo 20 registros)
        cursor.executemany("""
            INSERT INTO structured_logs (name, committees, distrit, party)
            VALUES (%s, %s, %s, %s);
        """, members[:total_mimbros])

        # Confirmar cambios y cerrar la conexión
        conn.commit()
        print("Datos estructurados insertados correctamente.")
        cursor.close()
        conn.close()

    except Exception as e:
        print(f"Error al insertar datos estructurados: {e}")

if __name__ == "__main__":
    create_database_and_table() # Crear base de datos y tabla
    insert_structured_logs() # Insertar datos estructurados
    # save_unstructured_logs() # Guardar logs no estructurados

La base de datos 'senadores_house_logs' ya existe.
Tabla 'structured_logs' creada correctamente.
Datos estructurados insertados correctamente.


## FIN BASE

In [ ]:
# Should be 61
# len(members)

118

Echemos un vistazo a lo que tenemos en  `members`.

In [64]:
print(members[:5])

[('Carol Ammons', 103, 'D', 'http://www.ilga.gov/house/RepBills.asp?MemberID=3309&Primary=True'), ('Jaime M. Andrade, Jr.', 40, 'D', 'http://www.ilga.gov/house/RepBills.asp?MemberID=3307&Primary=True'), ('Dagmara Avelar', 85, 'D', 'http://www.ilga.gov/house/RepBills.asp?MemberID=3391&Primary=True'), ('Harry Benton', 97, 'D', 'http://www.ilga.gov/house/RepBills.asp?MemberID=3432&Primary=True'), ('Diane Blair-Sherlock', 46, 'D', 'http://www.ilga.gov/house/RepBills.asp?MemberID=3425&Primary=True')]


## 🥊  Desafío: Obtener elementos `href` que apunten a los proyectos de ley de los miembros

El código anterior recupera información sobre:
- el nombre del senador
- su número de distrito,
- y su partido.

Ahora queremos recuperar la URL de la lista de proyectos de ley de cada senador. Cada URL seguirá un formato específico. 

El formato de la lista de proyectos de ley para un senador determinado es:

`http://www.ilga.gov/senate/SenatorBills.asp?GA=98&MemberID=[MEMBER_ID]&Primary=True`

para obtener algo como:

`http://www.ilga.gov/senate/SenatorBills.asp?MemberID=1911&GA=98&Primary=True`

en el que `MEMBER_ID=1911`. 

Debería poder ver que, desafortunadamente, `MEMBER_ID` ` no es actualmente algo que se extraiga en nuestro código de raspado.

Tu tarea inicial es modificar el código anterior para que también **recuperemos la URL completa que apunta a la página correspondiente de los proyectos de ley patrocinados por las primarias**, para cada miembro, y la devolvamos junto con su nombre, distrito y partido.

Consejos: 

* Para ello, querrás obtener el elemento de anclaje apropiado (`<a>`) en cada fila de legisladores de la tabla. Puedes utilizar de nuevo el método `.select()` en el objeto `row` en el bucle para hacer esto - similar al comando que encuentra todas las celdas `td.detail` en la fila. Recuerda que sólo queremos el enlace a los proyectos de ley del legislador, no a los comités ni a la página de perfil del legislador.
* El HTML de los elementos de anclaje tendrá el siguiente aspecto: `<a href="/senate/Senator.asp/...">Bills</a>`. La cadena en el atributo `href` contiene el enlace **relative** que buscamos. Puedes acceder a un atributo de un objeto BeatifulSoup `Tag` de la misma forma que accedes a un diccionario de Python: `anchor['attributeName']`. Véase el <a href="http://www.crummy.com/software/BeautifulSoup/bs4/doc/#tag">documentation</a> para más detalles.
* Hay un montón de maneras diferentes de utilizar BeautifulSoup para hacer las cosas. lo que usted necesita hacer para sacar el `href` está bien.

El código ha sido parcialmente rellenado para usted. Rellénalo donde dice `#YOUR CODE HERE`. Guarda la ruta en un objeto llamado `full_path`.

In [51]:
# Make a GET request
req = requests.get('https://www.ilga.gov/house/default.asp?GA=104')
# Read the content of the server’s response
src = req.text
# Soup it
soup = BeautifulSoup(src, "lxml")
# Create empty list to store our data
members = []

# Returns every ‘tr tr tr’ css selector in the page
rows = soup.select('tr tr tr')
# Get rid of junk rows
rows = [row for row in rows if row.select('td.detail')]

# Loop through all rows
for row in rows:
    # Select only those 'td' tags with class 'detail'
    detail_cells = row.select('td.detail') 
    # Keep only the text in each of those cells
    row_data = [cell.text for cell in detail_cells]
    # Collect information
    name = row_data[0]
    district = int(row_data[3])
    party = row_data[4]
    
    # YOUR CODE HERE
    # Extract href
    href = row.select('a')[1]['href']
    # Create full path
    full_path = "http://www.ilga.gov/house/" + href + "&Primary=True"
    
    # Store in a tuple
    senator = (name, district, party, full_path)
    # Append to list
    members.append(senator)

In [52]:
# Uncomment to test 
members[:5]

[('Carol Ammons',
  103,
  'D',
  'http://www.ilga.gov/house/RepBills.asp?MemberID=3309&Primary=True'),
 ('Jaime M. Andrade, Jr.',
  40,
  'D',
  'http://www.ilga.gov/house/RepBills.asp?MemberID=3307&Primary=True'),
 ('Dagmara Avelar',
  85,
  'D',
  'http://www.ilga.gov/house/RepBills.asp?MemberID=3391&Primary=True'),
 ('Harry Benton',
  97,
  'D',
  'http://www.ilga.gov/house/RepBills.asp?MemberID=3432&Primary=True'),
 ('Diane Blair-Sherlock',
  46,
  'D',
  'http://www.ilga.gov/house/RepBills.asp?MemberID=3425&Primary=True')]

## 🥊  Desafío: Modularice su código

Convierta el código anterior en una función que acepte una URL, rastree la URL en busca de sus senadores y devuelva una lista de tuplas con información sobre cada senador.

In [53]:
# YOUR CODE HERE
def get_members(url):
    # Make a GET request
    req = requests.get(url)
    # Read the content of the server’s response
    src = req.text
    # Soup it
    soup = BeautifulSoup(src, "lxml")
    # Create empty list to store our data
    members = []

    # Returns every ‘tr tr tr’ css selector in the page
    rows = soup.select('tr tr tr')
    # Get rid of junk rows
    rows = [row for row in rows if row.select('td.detail')]

    # Loop through all rows
    for row in rows:
        # Select only those 'td' tags with class 'detail'
        detail_cells = row.select('td.detail') 
        # Keep only the text in each of those cells
        row_data = [cell.text for cell in detail_cells]
        # Collect information
        name = row_data[0]
        district = int(row_data[3])
        party = row_data[4]

        # YOUR CODE HERE
        # Extract href
        href = row.select('a')[1]['href']
        # Create full path
        full_path = "https://www.ilga.gov/house/" + href + "&Primary=True"

        # Store in a tuple
        senator = (name, district, party, full_path)
        # Append to list
        members.append(senator)
    return(members)

In [54]:
# Test your code
url = 'https://www.ilga.gov/house/default.asp?GA=104'
senate_members = get_members(url)
len(senate_members)

118

## 🥊 Reto para llevar a casa: Escribir una función rascadora

Queremos raspar las páginas web correspondientes a los proyectos de ley patrocinados por cada uno de ellos.
Escribe una función llamada `get_bills(url)` para analizar una URL de facturas dada. Esto implicará:

  - solicitando la URL mediante la biblioteca <a href="http://docs.python-requests.org/en/latest/">`requests`</a>   - utilizando las características de la biblioteca `BeautifulSoup` para encontrar todos los elementos `<td>` con la clase `billlist`.
  - devuelve una _lista_ de tuplas, cada una con:
      - descripción (2ª columna)
      - cámara (S o H) (3ª columna)
      - la última acción (4ª columna)
      - la fecha de la última acción (5ª columna)
      
Esta función se ha completado parcialmente. Rellene el resto.

In [56]:
def get_bills(url):
    src = requests.get(url).text
    soup = BeautifulSoup(src)
    rows = soup.select('tr tr tr')
    bills = []
    # Iterate over rows
    for row in rows:
        # Grab all bill list cells
        cells = row.select('td.billlist')
        # Keep in mind the name of the senator is not a billlist class!
        if len(cells) == 5:
            row_text = [cell.text for cell in cells]
            # Extract info from row text
            bill_id = row_text[0]
            description = row_text[1]
            chamber = row_text[2]
            last_action = row_text[3]
            last_action_date = row_text[4]
            # Consolidate bill info
            bill = (bill_id, description, chamber, last_action, last_action_date)
            bills.append(bill)
    return bills

In [57]:
# Uncomment to test your code
test_url = senate_members[0][3]
get_bills(test_url)[0:5]

[('HB1581',
  'EQUITABLE UNIVERSITY FUNDING',
  'H',
  'Rule 19(a) / Re-referred to Rules Committee',
  '3/21/2025'),
 ('HB2466',
  'HIGHER ED-INCARCERATED STUDENT',
  'H',
  'Held on Calendar Order of Second Reading - Short Debate',
  '3/26/2025'),
 ('HB2764',
  'CD CORR-EARNED REENTRY',
  'H',
  'Rule 19(a) / Re-referred to Rules Committee',
  '3/21/2025'),
 ('HB3302',
  'EV REBATE ACT-ADMINISTRATION',
  'H',
  'Rule 19(a) / Re-referred to Rules Committee',
  '3/21/2025'),
 ('HB3355',
  'ELEC CD-VOTE BY MAIL',
  'H',
  'Rule 19(a) / Re-referred to Rules Committee',
  '3/21/2025')]

### Rechazar todas las facturas

Por último, crea un diccionario `bills_dict` que asigne un número de distrito (la clave) a una lista de proyectos de ley (el valor) procedentes de ese distrito. Puedes hacerlo recorriendo todos los miembros del senado en `members_dict` y llamando a `get_bills()` para cada una de sus URLs de proyectos de ley asociadas.

**NOTA:** por favor llama a la función `time.sleep(1)` en cada iteración del bucle, para que no destruyamos la web del estado.

In [58]:
# YOUR CODE HERE
bills_dict = {}
for member in senate_members[:5]:
    bills_dict[member[1]] = get_bills(member[3])
    time.sleep(1)

In [60]:
print(bills_dict)

{103: [('HB1581', 'EQUITABLE UNIVERSITY FUNDING', 'H', 'Rule 19(a) / Re-referred to Rules Committee', '3/21/2025'), ('HB2466', 'HIGHER ED-INCARCERATED STUDENT', 'H', 'Held on Calendar Order of Second Reading - Short Debate', '3/26/2025'), ('HB2764', 'CD CORR-EARNED REENTRY', 'H', 'Rule 19(a) / Re-referred to Rules Committee', '3/21/2025'), ('HB3302', 'EV REBATE ACT-ADMINISTRATION', 'H', 'Rule 19(a) / Re-referred to Rules Committee', '3/21/2025'), ('HB3355', 'ELEC CD-VOTE BY MAIL', 'H', 'Rule 19(a) / Re-referred to Rules Committee', '3/21/2025'), ('HB3356', 'HAIR BRAIDING LICENSURE REPEAL', 'H', 'Held on Calendar Order of Second Reading - Short Debate', '3/26/2025'), ('HB3357', '$ST UNI CIVIL SERV-OP EXPENSES', 'H', 'Assigned to Appropriations-Pensions & Personnel', '3/11/2025'), ('HB3459', 'EMPLOYMENT-32 HOUR WORK WEEK', 'H', 'Referred to Rules Committee', '2/18/2025'), ('HB3462', 'DFPR-CRIMINAL CONVICTIONS', 'H', 'Held on Calendar Order of Second Reading - Short Debate', '3/26/2025'),

In [61]:
# Uncomment to test your code
len(bills_dict[103])

12